In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [ ]:
#these families just had training rxns added to them 

#dict_keys(['1,2_Insertion_carbene', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement'])

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['R_Recombination', '1,2_Insertion_carbene', 'F_Abstraction', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', 'Intra_R_Add_Endocyclic', 'R_Addition_COm', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']

In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


1 *1 [CO,CS,Cdd,N] u0 {2,D}
2 *2 [O,N,S]       u0 {1,D}
3 *3 C2s           u0 p1 {4,S} {5,S}
4    [H,Val7]      u0 {3,S}
5    [H,Val7]      u0 {3,S}

1 *1 [Cd,Cdd,N] u0 {2,D}
2 *2 [Cd,Cdd,N] u0 {1,D}
3 *3 C2s        u0 p1 {4,S} {5,S}
4    [H,Val7]   u0 {3,S}
5    [H,Val7]   u0 {3,S}

1 *1 [Ct,N]   u0 {2,T}
2 *2 [Ct,N]   u0 {1,T}
3 *3 C2s      u0 p1 {4,S} {5,S}
4    [H,Val7] u0 {3,S}
5    [H,Val7] u0 {3,S}

1 *1 [CO,CS,Cdd,N] u0 {2,D}
2 *2 [O,N,S]       u0 {1,D}
3    Cs            u0 {4,S} {5,S} {6,S} {7,S}
4 *3 C2s           u0 p1 {3,S} {8,S}
5    [H,Val7]      u0 {3,S}
6    [H,Val7]      u0 {3,S}
7    [H,Val7]      u0 {3,S}
8    [H,Val7]      u0 {4,S}

1 *1 [Cd,Cdd,N] u0 {2,D}
2 *2 [Cd,Cdd,N] u0 {1,D}
3    Cs         u0 {4,S} {5,S} {6,S} {7,S}
4 *3 C2s        u0 p1 {3,S} {8,S}
5    [H,Val7]   u0 {3,S}
6    [H,Val7]   u0 {3,S}
7    [H,Val7]   u0 {3,S}
8    [H,Val7]   u0 {4,S}

1 *1 [Ct,N]   u0 {2,T}
2 *2 [Ct,N]   u0 {1,T}
3    Cs       u0 {4,S} {5,S} {6,S} {7,S}
4 *3 C2s      u0 p1 {3,

In [5]:
database.kinetics.families

{'1+2_Cycloaddition': <ReactionFamily "1+2_Cycloaddition">,
 '1,2_Insertion_CO': <ReactionFamily "1,2_Insertion_CO">,
 '1,2_Insertion_carbene': <ReactionFamily "1,2_Insertion_carbene">,
 '1,3_Insertion_CO2': <ReactionFamily "1,3_Insertion_CO2">,
 '1,3_sigmatropic_rearrangement': <ReactionFamily "1,3_sigmatropic_rearrangement">,
 'F_Abstraction': <ReactionFamily "F_Abstraction">,
 'Intra_R_Add_Endocyclic': <ReactionFamily "Intra_R_Add_Endocyclic">,
 'R_Addition_COm': <ReactionFamily "R_Addition_COm">,
 'R_Addition_MultipleBond': <ReactionFamily "R_Addition_MultipleBond">,
 'R_Recombination': <ReactionFamily "R_Recombination">,
 'Singlet_Carbene_Intra_Disproportionation': <ReactionFamily "Singlet_Carbene_Intra_Disproportionation">,
 'XY_Addition_MultipleBond': <ReactionFamily "XY_Addition_MultipleBond">}

In [6]:
# '1,3_Insertion_CO2' 
family_to_train = "1,3_Insertion_CO2"
family = database.kinetics.families[family_to_train]

In [7]:
family.clean_tree()

In [8]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

multiplicity -187
1 *3 H u0 p0 c0 {2,S}
2 *4 H u0 p0 c0 {1,S}
3 *2 O u0 p2 c0 {5,D}
4    O u0 p2 c0 {5,D}
5 *1 C u0 p0 c0 {3,D} {4,D}

multiplicity -187
1 *3 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 *4 H u0 p0 c0 {1,S}
3    H u0 p0 c0 {1,S}
4    H u0 p0 c0 {1,S}
5    H u0 p0 c0 {1,S}
6 *2 O u0 p2 c0 {8,D}
7    O u0 p2 c0 {8,D}
8 *1 C u0 p0 c0 {6,D} {7,D}

multiplicity -187
1     C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2  *3 C u0 p0 c0 {1,S} {6,S} {7,S} {8,S}
3     H u0 p0 c0 {1,S}
4     H u0 p0 c0 {1,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {2,S}
7  *4 H u0 p0 c0 {2,S}
8     H u0 p0 c0 {2,S}
9  *2 O u0 p2 c0 {11,D}
10    O u0 p2 c0 {11,D}
11 *1 C u0 p0 c0 {9,D} {10,D}

multiplicity -187
1  *3 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2     C u0 p0 c0 {1,S} {6,S} {7,S} {8,S}
3     C u0 p0 c0 {1,S} {9,S} {10,S} {11,S}
4  *4 H u0 p0 c0 {1,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {2,S}
7     H u0 p0 c0 {2,S}
8     H u0 p0 c0 {2,S}
9     H u0 p0 c0 {3,S}
10    H u0 p0 c0 {3,S}
11    H u0 p0 c0 {3,S}
12

InvalidActionError: Attempted to remove a nonexistent bond.

In [ ]:
len(family.groups.entries)

In [ ]:
family.groups.entries

In [ ]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

In [ ]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

In [ ]:
len(templateRxnMap)

In [ ]:
family.clean_tree_rules()

In [ ]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

In [ ]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

In [ ]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

In [ ]:
family.save(save_path)

In [ ]:
# '1,2_Insertion_CO'

In [ ]:
# '1,2_Insertion_carbene'

In [ ]:
# '1,3_Insertion_CO2'

In [ ]:
# '1,3_sigmatropic_rearrangement'

In [ ]:
# 'R_Addition_MultipleBond'

In [ ]:
# 'Singlet_Carbene_Intra_Disproportionation'

In [ ]:
#'XY_Addition_MultipleBond'

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(np.abs(list(errors.values())),bins=30, density=True)
plt.title('Decision Tree Estimator',fontsize=18)
#plt.ylim(0,.4)
plt.xlim(0,15)
plt.ylabel('Probability density',fontsize=18)
plt.xlabel(r'$|Ln(k_{est}/k_{rxn})|$',fontsize=18)